In [ ]:
%ntbl pull datasets "hh_ru"

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv(r"../datasets/hh_ru/hh_ru_dataset.csv")
df.info()

In [105]:
df_cleared = df[::].copy()
len(df_cleared)


In [ ]:
len(df_cleared[df_cleared['compensation_to'] == 0])

In [106]:
quantile_param = 0.04
quantiles = {}
for prof in df['profession'].unique():
    df_prof = df_cleared[df_cleared['profession'] == prof]
    quantiles[prof] = (df_prof['expected_salary'].quantile(quantile_param),
                       df_prof['expected_salary'].quantile(1 - quantile_param))

df_cleared.loc[:,'exp_salary_min_limit'] = [quantiles[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'exp_salary_max_limit'] = [quantiles[i][1] for i in df_cleared['profession']]


In [ ]:
df_cleared = df_cleared[df_cleared['compensation_to'] != 0]
len(df_cleared)

In [113]:
quantile_param = 0.01
quantiles = {}
for prof in df['profession'].unique():
    df_prof = df_cleared[df_cleared['profession'] == prof]
    quantiles[prof] = (df_prof['compensation_to'].quantile(quantile_param),
                       df_prof['compensation_to'].quantile(1 - quantile_param))
df_cleared.loc[:,'compensation_to_min_limit'] = [quantiles[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'compensation_to_max_limit'] = [quantiles[i][1] for i in df_cleared['profession']]

df_cleared = df_cleared[(df_cleared['compensation_to'] <=
                         df_cleared['compensation_to_max_limit']) &
                       (df_cleared['compensation_to'] >=
                        df_cleared['compensation_to_min_limit'])]

len(df_cleared)
#df_cleared['compensation_to'].sort_values().head(10)

In [107]:
df_cleared = df_cleared[(df_cleared['expected_salary'] >= df_cleared['exp_salary_min_limit']) &
                        (df_cleared['expected_salary'] <= df_cleared['exp_salary_max_limit'])]
len(df_cleared)

In [108]:
df_cleared = df_cleared[(df.initial_state != 'discard') & ((df.final_state == 'invitation') | (df.final_state == 'discard'))]
# df_cleared = df_cleared[df.compensation_from <= df.compensation_to]
len(df_cleared)

In [109]:
df_cleared = df_cleared.drop_duplicates(['topic_id'])
df_cleared = df_cleared.dropna(axis=0, subset=['education_level', 'initial_state', 'final_state', 'work_schedule', 'compensation_from', 'compensation_to'])
len(df_cleared)

In [111]:
# df_cleared = df_cleared[(df_cleared['compensation_to'] <= df_cleared['compensation_to_max_limit'])]
df_cleared = df_cleared[(2023 - df_cleared.work_experience_months / 12 - df_cleared.year_of_birth >= 18) & (df_cleared.work_experience_months <= 600)]
df_cleared = df_cleared[(df_cleared['year_of_birth'] >= 1953) &
                        (df_cleared['year_of_birth'] <= 2005)]
len(df_cleared)
# df_cleared['compensation_to'].sort_values()

In [112]:
df_cleared = df_cleared[df_cleared.compensation_from <= df_cleared.compensation_to]
len(df_cleared)

In [ ]:
gender_dict = {'female': 0, 'male': 1}
state_dict = {'discard': 0, 'invitation': 1}
education_dict = {'secondary': 1, 'special_secondary': 2, 'unfinished_higher': 3,
                 'bachelor': 4, 'master': 5, 'higher': 6, 'candidate': 7, 'doctor': 8}
df_cleared = df_cleared.replace({'gender': gender_dict, 'education_level': education_dict,
                                'final_state': state_dict})

In [ ]:
df_cleared.sort_values('compensation_to').head(20)

In [95]:
df_cleared.to_excel(r'clear5.xlsx')

In [ ]:
len(df[(df.education_level == 'special_secondary') & 
   (df.compensation_from <= df.compensation_to) & 
   ((df.year_of_birth >= 1953) & (df.year_of_birth <= 2005)) & 
  ((2023 - df.work_experience_months / 12 - df.year_of_birth >= 18) & (df.work_experience_months <= 600)) &
  (df.initial_state != 'discard') & ((df.final_state == 'invitation') | (df.final_state == 'discard')) & 
      (df_cleared.compensation_to == 0)])

In [ ]:
len(df[(df.education_level == 'special_secondary') & (2023 - df.work_experience_months / 12 - df.year_of_birth < 18) & (df.work_experience_months > 600)])

In [ ]:
df_cleared.info()